In [ ]:
!pip -q install pandas numpy scikit-learn matplotlib torch --extra-index-url https://download.pytorch.org/whl/cpu


In [ ]:
import os, zipfile, shutil, random
import numpy as np, pandas as pd
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

# Reproducibility
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)

# Data & output dirs
BASE = Path("/content/data")
CERT_DIR, LANL_DIR, ENRON_DIR = BASE/"CERT", BASE/"LANL", BASE/"ENRON"
for p in [CERT_DIR, LANL_DIR, ENRON_DIR]:
    p.mkdir(parents=True, exist_ok=True)

OUT_DIR = Path("/content/output")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Config
ROW_LIMIT_CERT = None    # e.g. 50_000 for quick tests
ROW_LIMIT_LANL = None
ROW_LIMIT_ENRON = None
VAL_PCTL = 95            # AE threshold percentile on validation


In [ ]:
# Upload a single ZIP with folders CERT/, LANL/, ENRON/ or upload individual CSVs
try:
    from google.colab import files
    print("Upload a ZIP with CERT/, LANL/, ENRON/ or individual CSVs (logon.csv, file.csv, device.csv, auth.csv, emails.csv)")
    uploaded = files.upload()

    for name, data in uploaded.items():
        p = BASE/name
        with open(p, "wb") as f: f.write(data)
        if zipfile.is_zipfile(p):
            print("Extracting:", name)
            with zipfile.ZipFile(p) as z: z.extractall(BASE)

    # Move loose CSVs into their folders (if needed)
    for name in uploaded.keys():
        src = BASE/name
        lname = name.lower()
        if lname.endswith("logon.csv"):     shutil.move(str(src), str(CERT_DIR/"logon.csv"))
        elif lname.endswith("file.csv"):    shutil.move(str(src), str(CERT_DIR/"file.csv"))
        elif lname.endswith("device.csv"):  shutil.move(str(src), str(CERT_DIR/"device.csv"))
        elif lname.endswith("auth.csv"):    shutil.move(str(src), str(LANL_DIR/"auth.csv"))
        elif lname.endswith("emails.csv"):  shutil.move(str(src), str(ENRON_DIR/"emails.csv"))
except Exception:
    print("[INFO] Not in Colab; ensure files already exist under /content/data.")

print("CERT files:", [p.name for p in CERT_DIR.glob("*.csv")])
print("LANL files:", [p.name for p in LANL_DIR.glob("*.csv")])
print("ENRON files:", [p.name for p in ENRON_DIR.glob("*.csv")])


Upload a ZIP with CERT/, LANL/, ENRON/ or individual CSVs (logon.csv, file.csv, device.csv, auth.csv, emails.csv)


Saving Cert.csv to Cert.csv
Saving Enron.csv to Enron.csv
Saving Lan.csv to Lan.csv
CERT files: []
LANL files: []
ENRON files: []
